# **Assignment 05 - Random search and hyperopt**

## Importing Libraries

In [1]:
import pandas as pd
import numpy as np

## Loading titanic.csv file

In [2]:
df = pd.read_csv('titanic.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


## Selecting 'Pclass','SibSp','Parch','Fare' columns as data and ‘Survived’ column as label 

In [5]:
X = df[['Pclass','SibSp','Parch','Fare']]
y = df.Survived

In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  891 non-null    int64  
 1   SibSp   891 non-null    int64  
 2   Parch   891 non-null    int64  
 3   Fare    891 non-null    float64
dtypes: float64(1), int64(3)
memory usage: 28.0 KB


In [7]:
pd.DataFrame(y).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Survived  891 non-null    int64
dtypes: int64(1)
memory usage: 7.1 KB


In [8]:
y.value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

## **Applying SVM to classify the data** **bold text**

In [10]:
from sklearn.svm import SVC
svm_classifier = SVC(kernel = 'linear', random_state = 0)
svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict(X_test)

### **5-Fold Cross Validation to generalise the model accuracy**

In [11]:
from sklearn.model_selection import cross_val_score,KFold
svm_classifier = SVC(kernel = 'linear', random_state = 0)
kf=KFold(n_splits=5)
score=cross_val_score(svm_classifier,X,y,cv=kf)
print("Cross Validation Scores are {}".format(score))

Cross Validation Scores are [0.61452514 0.65730337 0.66292135 0.7247191  0.74719101]


In [12]:
print("Average Cross Validation score :{}".format(score.mean()))

Average Cross Validation score :0.6813319942250958


## **Use random search to identify the best possible SVM model**

### Import RandomizedSearchCV

In [13]:
from sklearn.model_selection import RandomizedSearchCV

### Checking the list of available parameters

In [14]:
SVC().get_params().keys()

dict_keys(['C', 'break_ties', 'cache_size', 'class_weight', 'coef0', 'decision_function_shape', 'degree', 'gamma', 'kernel', 'max_iter', 'probability', 'random_state', 'shrinking', 'tol', 'verbose'])

In [15]:
parameters ={'C': [0.1, 1, 10, 100, 1000],
'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
'kernel': ['rbf']}

### Random search on Hyperparameters

In [16]:
clf = RandomizedSearchCV(SVC(), parameters, random_state=0)

In [17]:
X_train.shape, y_train.shape

((668, 4), (668,))

In [18]:
X_test.shape, y_test.shape

((223, 4), (223,))

In [19]:
clf.fit(X_train, y_train)

RandomizedSearchCV(estimator=SVC(),
                   param_distributions={'C': [0.1, 1, 10, 100, 1000],
                                        'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                                        'kernel': ['rbf']},
                   random_state=0)

### Checking the best parameter

In [20]:
clf.best_params_

{'kernel': 'rbf', 'gamma': 0.01, 'C': 100}

## **Use hyperopt to identify the best possible SVM model**


### Install Hyperopt

In [21]:
pip install hyperopt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### import important packages include Hyperopt

In [22]:
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials
from hyperopt.pyll.base import scope
import warnings
warnings.filterwarnings("ignore")

### Define Parameter Space for Optimization

In [26]:
space = {
    'C': hp.choice('C',[0.1, 1, 10, 100, 1000]),
    'gamma': hp.choice('gamma',[1, 0.1, 0.01, 0.001, 0.0001]),
    'kernel': hp.choice('kernel',['rbf']),
    }

### Defining hyperparamter_tuning and using SVM classification algorithm to optimize hyperparameter

In [24]:
def hyperparameter_tuning(params):
    svm_classifier = SVC(kernel = 'linear', random_state = 0)
    acc = cross_val_score(svm_classifier, X, y,scoring="accuracy").mean()
    return {"loss": -acc, "status": STATUS_OK}

### Fine tune the model

In [25]:
# Initialize trials object
trials = Trials()

best = fmin(
    fn=hyperparameter_tuning,
    space = space, 
    algo=tpe.suggest, 
    max_evals=100, 
    trials=trials
)

print("Best: {}".format(best))

100%|██████████| 100/100 [1:02:11<00:00, 37.32s/it, best loss: -0.6858389303872953]
Best: {'C': 1, 'gamma': 1, 'kernel': 0}


### Analyzing results by trials

In [27]:
trials.results

[{'loss': -0.6858389303872953, 'status': 'ok'},
 {'loss': -0.6858389303872953, 'status': 'ok'},
 {'loss': -0.6858389303872953, 'status': 'ok'},
 {'loss': -0.6858389303872953, 'status': 'ok'},
 {'loss': -0.6858389303872953, 'status': 'ok'},
 {'loss': -0.6858389303872953, 'status': 'ok'},
 {'loss': -0.6858389303872953, 'status': 'ok'},
 {'loss': -0.6858389303872953, 'status': 'ok'},
 {'loss': -0.6858389303872953, 'status': 'ok'},
 {'loss': -0.6858389303872953, 'status': 'ok'},
 {'loss': -0.6858389303872953, 'status': 'ok'},
 {'loss': -0.6858389303872953, 'status': 'ok'},
 {'loss': -0.6858389303872953, 'status': 'ok'},
 {'loss': -0.6858389303872953, 'status': 'ok'},
 {'loss': -0.6858389303872953, 'status': 'ok'},
 {'loss': -0.6858389303872953, 'status': 'ok'},
 {'loss': -0.6858389303872953, 'status': 'ok'},
 {'loss': -0.6858389303872953, 'status': 'ok'},
 {'loss': -0.6858389303872953, 'status': 'ok'},
 {'loss': -0.6858389303872953, 'status': 'ok'},
 {'loss': -0.6858389303872953, 'status':

In [28]:
trials.losses()

[-0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.6858389303872953,
 -0.685838

In [29]:
trials.statuses()

['ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok']